In [1]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt 
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
from sklearn.linear_model import LinearRegression
import task1 as t1 
import task2 as t2 
import task3 as t3 
from scipy.stats import spearmanr

In [2]:

data_root_min = '/Users/huayuzhu/Desktop/exam/raw_data/minute'
data_root = '/Users/huayuzhu/Desktop/exam/raw_data/daily'
output_dir = '/Users/huayuzhu/Desktop/exam/'
def get_min_data_from_csv(file_path):
    """
    Reads a CSV file into a DataFrame, with the first column as row indices.

    Parameters
    ----------
    file_path : str
        The file path of the CSV file to be read.

    Returns
    -------
    pd.DataFrame
        Transposed DataFrame with datetime as index.
    """
    try:
        data = pd.read_csv(file_path, index_col=0) 
        data.index = pd.to_datetime(data.index)
    except FileNotFoundError:
        print(f"File '{file_path}' not found.")
    except Exception as e:
        print(f"An error occurred: {e}")
    return data
def get_data_from_csv(file_path):
    """
    Reads a CSV file into a DataFrame, with the first column as row indices, 
    transposes the result, and converts the row indices to datetime.

    Parameters
    ----------
    file_path : str
        The file path of the CSV file to be read.

    Returns
    -------
    pd.DataFrame
        Transposed DataFrame with datetime as index.
    """
    try:
        data = pd.read_csv(file_path, index_col=0).T
        data.index = pd.to_datetime(data.index)
    except FileNotFoundError:
        print(f"File '{file_path}' not found.")
    except Exception as e:
        print(f"An error occurred: {e}")
    return data
amount = get_min_data_from_csv(f'{data_root_min}/amount.csv')
volume = get_min_data_from_csv(f'{data_root_min}/volume.csv')
close  = get_min_data_from_csv(f'{data_root_min}/close.csv')
open  = get_min_data_from_csv(f'{data_root_min}/open.csv')

S_DQ_RET = get_data_from_csv(f'{data_root}/S_DQ_RET.csv')
S_905_DQ_RET =  get_data_from_csv(f'{data_root}/905S_DQ_RET.csv')
S_DQ_MV = get_data_from_csv(f'{data_root}/S_DQ_MV.csv')
S_RESTRICT = get_data_from_csv(f'{data_root}/S_RESTRICT.csv')
S_DQ_OPEN = get_data_from_csv(f'{data_root}/S_DQ_OPEN.csv')
S_DQ_ADJ_FACTOR = get_data_from_csv(f'{data_root}/S_DQ_ADJFACTOR.csv')
S_DQ_CLOSE = get_data_from_csv(f'{data_root}/S_DQ_CLOSE.csv')
S_DQ_VOLUME = get_data_from_csv(f'{data_root}/S_DQ_VOLUME.csv')
FLOAT_A_SHR_TODAY = get_data_from_csv(f'{data_root}/FLOAT_A_SHR_TODAY.csv')

In [3]:
S_DQ_ADJ_FACTOR = S_DQ_ADJ_FACTOR/100
S_ADJ_CLOSE = S_DQ_CLOSE * S_DQ_ADJ_FACTOR
S_ADJ_OPEN = S_DQ_OPEN * S_DQ_ADJ_FACTOR

In [63]:
def illiquidty_factor(open,close,amount,t):
    returns = close/ open.shift(1) - 1
    ret_num = returns.fillna(0)
    amount_den = amount.fillna(0)

    def calc_illiq(x):
        cumulative_product = (1 + abs(x)).prod()
        return np.log(cumulative_product)

    illi_factor_numerator = ret_num.rolling(window=t).apply(calc_illiq, raw=True)
    illi_factor_denominator = amount_den.rolling(window=t).sum()
    illi_factor = illi_factor_numerator / illi_factor_denominator

    return illi_factor[t-1:]

In [67]:
close.columns

Index(['000001', '000002', '000009', '000027', '000063', '000066', '000069',
       '000100', '000157', '000166',
       ...
       '688122', '688126', '688180', '688188', '688363', '688390', '688561',
       '688599', '688777', '688981'],
      dtype='object', length=500)

In [69]:
S_DQ_CLOSE.columns 

Index([     1,      2,      3,      4,      5,      6,      7,      8,      9,
           10,
       ...
       873593, 873665, 873679, 873690, 873693, 873703, 873726, 873749, 873806,
       873833],
      dtype='int64', length=5566)

In [64]:
illiquidty_factor(open,close,amount,5)

,000001,000002,000009,000027,000063,000066,000069,000100,000157,000166,...,688122,688126,688180,688188,688363,688390,688561,688599,688777,688981
minute_idx,,,,,,,,,,,,,,,,,,,,,
2021-01-04 09:36:00,2.015100e-10,1.147596e-10,8.581189e-10,8.723394e-10,1.232977e-10,1.378630e-10,7.789566e-10,1.433627e-10,2.958457e-10,6.575338e-10,...,5.542459e-10,5.721618e-10,1.546329e-09,5.076983e-09,1.991320e-09,1.908386e-09,9.869044e-10,5.917660e-10,2.400197e-09,3.664428e-10
2021-01-04 09:37:00,1.384190e-10,9.090711e-11,8.512284e-10,6.441183e-10,9.319177e-11,2.167345e-10,5.131221e-10,1.788108e-10,2.357700e-10,4.613479e-10,...,7.152378e-10,7.525313e-10,1.221983e-09,3.189136e-09,1.439718e-09,1.530040e-09,8.823008e-10,9.328641e-10,2.073078e-09,3.657781e-10
2021-01-04 09:38:00,1.443700e-10,8.122877e-11,4.511275e-10,7.739386e-10,1.165614e-10,2.375987e-10,6.380388e-10,1.910001e-10,2.403622e-10,3.804213e-10,...,8.427587e-10,8.467245e-10,1.061359e-09,2.971580e-09,1.303967e-09,1.326166e-09,1.549398e-09,9.378961e-10,2.810618e-09,3.152622e-10
2021-01-04 09:39:00,1.369712e-10,8.695670e-11,8.937715e-10,5.619267e-10,9.743551e-11,2.527051e-10,5.875133e-10,2.106094e-10,2.056968e-10,2.416195e-10,...,1.076626e-09,7.086064e-10,6.377055e-10,1.604686e-09,1.372626e-09,1.039887e-09,1.697897e-09,1.316573e-09,2.970413e-09,2.422528e-10
2021-01-04 09:40:00,7.469943e-11,9.143100e-11,9.353903e-10,4.639656e-10,1.091702e-10,2.556948e-10,3.705090e-10,1.556954e-10,2.139381e-10,2.776071e-10,...,8.907543e-10,3.012923e-10,5.905395e-10,9.421964e-10,1.687869e-09,1.414275e-09,2.009720e-09,1.613931e-09,2.851802e-09,2.950797e-10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-12-30 14:56:00,1.654309e-10,1.239778e-10,3.599187e-10,6.004965e-10,1.411923e-10,8.023167e-10,9.821307e-10,0.000000e+00,2.168167e-09,1.978051e-09,...,1.202107e-09,3.122497e-09,2.990791e-10,3.690911e-09,1.246416e-09,4.428067e-10,3.039234e-09,6.066945e-10,1.581440e-09,1.918707e-10
2022-12-30 14:57:00,1.115763e-10,6.647546e-11,5.555469e-10,1.100416e-09,2.063662e-10,6.510837e-10,1.319662e-09,3.591860e-10,3.233696e-09,1.808657e-09,...,1.419702e-09,2.740703e-09,2.545940e-10,6.263111e-09,9.662772e-10,3.221757e-10,3.125454e-09,6.576979e-10,2.127152e-09,1.265971e-10
2022-12-30 14:58:00,8.735541e-11,3.931912e-11,5.113646e-10,2.267163e-09,2.271551e-10,7.308992e-10,1.544664e-09,4.130006e-10,3.211194e-09,2.278896e-09,...,1.201416e-09,2.691170e-09,1.531822e-10,5.058634e-09,1.170044e-09,2.980858e-10,3.560492e-09,6.225247e-10,2.632143e-09,9.545185e-11


In [70]:
illi = illiquidty_factor(open,close,amount,5)

In [72]:
open_price = open.resample('D').mean() 
illi_factor = illi.resample('D').mean() 

In [81]:
class TestInfo: 
    def __init__(self, group_number = 10, trading_frequency = 'W', initial_capital = None, trading_cost =0.0):
        """
        Parameters:
        - group_number(int): Number of groups to divide into (default 10 for deciles).
        - trading_frequency(str): "M" (monthly), "W" (weekly) (default to weekly)
        - initial_capital(float): initial capital plan to invest, default to the capital needed at first period trading 
        - trading cost(float): trading cost percentage, default to 0 

        """
        self.group_number = group_number 
        self.trading_frequency = trading_frequency
        self.initial_capital = initial_capital 
        self.trading_cost = trading_cost 

class OneFactorTest: 
    """
    A class to conduct single factor testing for stock trading strategies. It ranks stocks, handles trade intervals,
    executes trades, and evaluates performance metrics against a benchmark.

    Attributes
    ----------
    factor : DataFrame
        Factor data used to rank stocks.
    stock_price_open : DataFrame
        Stock opening prices.
    restricted_stock_df : DataFrame
        Information on stocks that are restricted from trading.
    benchmark : DataFrame
        Benchmark data for comparison.

    Methods
    -------
    rank_stock(test_info):
        Ranks stocks based on the provided factor values.

    trade_interval(df, test_info):
        Resamples the given dataframe to the specified trading frequency and forward fills missing data.

    trade(test_info):
        Executes trades based on stock rankings and calculates performance.

    evaluation_metrics(performance, adj1=48, var=0.05):
        Calculates various performance metrics like attribution analysis and risk analysis metrics.

    eval_combined(test_info):
        Evaluates and combines performance metrics for top and bottom ranked stocks.

    compare_with_benchmark(test_info, name, output=False):
        Compares the strategy's performance with a benchmark, returns a dataframe of excess returns.

    plot_comparison_with_benchmark(test_info, name):
        Generates a plot comparing the daily and cumulative excess returns against a benchmark.
    """

    def __init__(self,test_factor,open):
        min_date = max(test_factor.index.min(), open.index.min())
        max_date = min(test_factor.index.max(), open.index.max())

        self.factor = test_factor.loc[min_date:max_date]
        self.stock_price_open = open.loc[min_date:max_date]
        #self.restricted_stock_df = S_RESTRICT
        self.benchmark = S_905_DQ_RET.loc[min_date:max_date]
    
    def rank_stock(self,test_info): 
        factor_value = self.factor.copy()
        percentile = factor_value.rank(axis=1, pct=True)
        group_number = test_info.group_number - ((1 - percentile) * test_info.group_number) // 1
        return group_number
    
    def trade_interval(self,df,test_info):
        df_factor = df.resample(test_info.trading_frequency).first()   
        df_factor = df_factor.ffill() 
        return df_factor

    def trade(self,test_info): 
        df = self.rank_stock(test_info)
        df_price = self.stock_price_open

        # decide initial capital to trade 
        if test_info.initial_capital is not None: 
            initial_k = test_info.initial_capital
        else:
            p_1_top = df.iloc[0] == 1.0
            p_1_bottom = df.iloc[0] == 10.0
            p_1_top_capital = df_price.iloc[0][p_1_top].sum()
            p_1_bottom_capital = df_price.iloc[0][p_1_bottom].sum()
            initial_k = 1000 * (p_1_bottom_capital + p_1_top_capital)
        side_k = initial_k / 2
        top_pnl_tracker = {'total_pnl': [0], 'cumulative_pnl': [0]}
        top_return_tracker = {'return': [0], 'net_return': [0]}
        bottom_pnl_tracker = {'total_pnl': [0], 'cumulative_pnl': [0]}
        bottom_return_tracker = {'net_return': [0]}
        
        factor_resample = self.trade_interval(df,test_info)
        price_resample = self.trade_interval(df_price, test_info)
        top_r = factor_resample.copy() 
        bottom_r = factor_resample.copy() 
        
        if (factor_resample.index != price_resample.index).any():
            print('please debug')
        for i, (date, signals) in enumerate(factor_resample.iterrows()):
            # here equal weighted portfolio strategy is used, Markovitz, or CAPM optimization may give better result  
            top = signals == 1.0
            bottom = signals == 10.0
            top_k = price_resample.loc[date][top].sum()
            bottom_k = price_resample.loc[date][bottom].sum()
            top_ratio = side_k / top_k if top_k else 0
            bottom_ratio = side_k / bottom_k if bottom_k else 0
            top_r.loc[date] = signals.apply(lambda x: top_ratio if x == 1 else 0)
            bottom_r.loc[date] = signals.apply(lambda x: bottom_ratio if x == 10 else 0)

            # profit calculation 
            if i > 0: 
                price_change = price_resample.iloc[i] - price_resample.iloc[i-1]
                
                top_pnl = (top_r.iloc[i-1] * price_change).sum()
                bottom_pnl = (bottom_r.iloc[i-1] * price_change).sum()
        
                top_cumulative_pnl = top_pnl_tracker['cumulative_pnl'][i-1] + top_pnl
                bottom_cumulative_pnl = bottom_pnl_tracker['cumulative_pnl'][i-1] + bottom_pnl

                top_pnl_tracker['total_pnl'].append(top_pnl)
                top_pnl_tracker['cumulative_pnl'].append(top_cumulative_pnl)

                bottom_pnl_tracker['total_pnl'].append(bottom_pnl)
                bottom_pnl_tracker['cumulative_pnl'].append(bottom_cumulative_pnl)

                top_return_tracker['net_return'].append(top_pnl/side_k - test_info.trading_cost)
                bottom_return_tracker['net_return'].append(bottom_pnl/side_k - test_info.trading_cost)
        df_top_pnl_tracker = pd.DataFrame(top_pnl_tracker, index=factor_resample.index)
        df_top_return_tracker = pd.DataFrame(top_return_tracker, index=factor_resample.index)
        df_top_performance = pd.concat([df_top_pnl_tracker, df_top_return_tracker], axis=1)
        df_top_performance['cumulative_return'] = ((df_top_performance['net_return'] + 1).cumprod() -1)/100

        df_bottom_pnl_tracker = pd.DataFrame(bottom_pnl_tracker, index=factor_resample.index)
        df_bottom_return_tracker = pd.DataFrame(bottom_return_tracker, index=factor_resample.index)
        df_bottom_performance = pd.concat([df_bottom_pnl_tracker, df_bottom_return_tracker], axis=1)
        df_bottom_performance['cumulative_return'] = ((df_bottom_performance['net_return'] + 1).cumprod() -1)/100

        return df_top_performance, df_bottom_performance
    
    def evaluation_mectrics(self,performance,adj1 = 48,var=0.05): 
        summary = dict()
        data = performance.loc[:,['net_return']].dropna()
        summary["Annualized Return"] = data.mean() * adj1
        summary["Annualized Volatility"] = data.std() * np.sqrt(adj1)
        summary["Annualized Sharpe Ratio"] = (
            summary["Annualized Return"] / summary["Annualized Volatility"]
        )
        summary["Annualized Sortino Ratio"] = summary["Annualized Return"] / (
            data[data < 0].std() * np.sqrt(adj1)
        )

        summary["Skewness"] = data.skew()
        summary["Excess Kurtosis"] = data.kurtosis()
        summary[f"VaR ({var})"] = data.quantile(var, axis=0)
        summary[f"CVaR ({var})"] = data[data <= data.quantile(var, axis=0)].mean()
        summary["Min"] = data.min()
        summary["Max"] = data.max()

        wealth_index = 1000 * (1 + data).cumprod()
        previous_peaks = wealth_index.cummax()
        drawdowns = (wealth_index - previous_peaks) / previous_peaks

        summary["Max Drawdown"] = drawdowns.min()

        summary["Bottom"] = drawdowns.idxmin()
        summary["Peak"] = previous_peaks.idxmax()
        return pd.DataFrame(summary) 
    
    def eval_combined(self,test_info): 
        per_top, per_bottom = self.trade(test_info)
        top = self.evaluation_mectrics('top',per_top)
        bottom = self.evaluation_mectrics('bottom',per_bottom)
        return pd.concat([top,bottom], axis=0,keys=['top', 'bottom'])
    
    def compare_with_benchmark(self,test_info,name,output = False):
        per_top, per_bottom = self.trade(test_info)
        ret_top = per_top.loc[:,['net_return']].dropna()
        ret_bottom = per_bottom.loc[:,['net_return']].dropna()
 

        daily_resampled_top = ret_top.resample('D').ffill().div(5)
        daily_resampled_bottom = ret_bottom.resample('D').ffill().div(5)
        daily_resampled_top = daily_resampled_top.reindex(self.benchmark.index, method='ffill')  
        daily_resampled_bottom = daily_resampled_bottom.reindex(self.benchmark.index, method='ffill') 

        daily_resampled_bottom.fillna(0, inplace= True)
        daily_resampled_top.fillna(0, inplace = True)

        if (self.benchmark.index != daily_resampled_bottom.index).any(): 
            print('please debug')
        summary = pd.DataFrame(index = self.benchmark.index) 
        summary["Daily excess_return TOP"] = (daily_resampled_top['net_return'] - self.benchmark[905])
        summary["Cumulative excess_return TOP"] = summary['Daily excess_return TOP'].cumsum()
        summary["Daily excess_return BOTTOM"] = (daily_resampled_bottom['net_return'] - self.benchmark[905])
        summary["Cumulative excess_return BOTTOM"] = summary['Daily excess_return BOTTOM'].cumsum()

        if output:
            summary.to_csv(f'{output_dir}/{name}_compairson_ret.csv')

        return summary
    
    def plot_comparison_with_benchmark(self, test_info,name):
        df = self.compare_with_benchmark(test_info,name)
        fig = make_subplots(specs=[[{"secondary_y": True}]])

        fig.add_trace(
            go.Scatter(x=df.index, y=df['Daily excess_return TOP'], name='Top Daily'),
            secondary_y=False,
        )

        fig.add_trace(
            go.Scatter(x=df.index, y=df['Daily excess_return BOTTOM'], name='Bottom Daily'),
            secondary_y=False,
        )


        fig.add_trace(
            go.Scatter(x=df.index, y=df['Cumulative excess_return TOP'], name='Top Cumulative'),
            secondary_y=True,
        )

        fig.add_trace(
            go.Scatter(x=df.index, y=df['Cumulative excess_return BOTTOM'], name='Bottom Cumulative'),
            secondary_y=True,
        )
       
        fig.update_layout(
            title_text=name+"Comparison of Daily and Cumulative Excess Returns",
            plot_bgcolor='white',  
            xaxis_showgrid=False,  
            yaxis_showgrid=False,  
            yaxis2_showgrid=False,  
            autosize=True,  
            template='plotly_white',
            colorway=px.colors.qualitative.Vivid
        )

        fig.show()



In [82]:
test1 = TestInfo()
factor3 = OneFactorTest(illi_factor,open_price)
factor3.plot_comparison_with_benchmark(test1,'factor3')
factor3.compare_with_benchmark(test1,'factor3',False)

,Daily excess_return TOP,Cumulative excess_return TOP,Daily excess_return BOTTOM,Cumulative excess_return BOTTOM
2021-01-04,-0.011670,-0.011670,-0.011670,-0.011670
2021-01-05,-0.012284,-0.023954,-0.012284,-0.023954
2021-01-06,0.003978,-0.019976,0.003978,-0.019976
2021-01-07,-0.002741,-0.022717,-0.002741,-0.022717
2021-01-08,-0.002430,-0.025147,-0.002430,-0.025147
...,...,...,...,...
2022-12-26,-0.015877,-0.191899,-0.027889,0.691350
2022-12-27,-0.002502,-0.194400,-0.014514,0.676836
2022-12-28,0.003842,-0.190558,-0.008170,0.668667
2022-12-29,-0.005866,-0.196424,-0.017878,0.650788
